# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import numpy as np
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sqlalchemy import create_engine
import re
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
%matplotlib inline

[nltk_data] Downloading package punkt to /Users/pepe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/pepe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/pepe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///db_disaster_response.db')
df = pd.read_sql_table('disaster_messages', con=engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df["message"]
y = df[df.columns[4:]]# We exclude the first 4 columns (id, message, original, genre)
category_names = y.columns

In [4]:
y.shape, X.shape

((26216, 36), (26216,))

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """tokenize and transform input text. Return cleaned text"""
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # take out all punctuation while tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    # lemmatize as shown in the lesson
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
    return clean_tokens

# Partially inspired from here: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#wordnetlemmatizer


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# We need to verify that the words are being tokenized (This willl not be included in the final script) 

# words_vector = CountVectorizer(tokenizer=tokenize)
# X_vectorized = words_vector.fit_transform(X)

# keys, values = [], []
# for k, v in words_vector.vocabulary_.items():
#     keys.append(k)
#     values.append(v)

# df_words = pd.DataFrame.from_dict({'words': keys, 'count': values})
# df_words
# plt.figure(figsize=(16, 6))
# sns.barplot(x="words", y="count", data=df_words)

In [7]:
MOC = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MOC)
])

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a19537830>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
# We split the test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20972,), (5244,), (20972, 36), (5244, 36))

In [10]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [11]:
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
def performance_results(y_test, y_pred, category_names):
    num = 0
    categories, f_scores, precisions, recalls = [], [], [], []
    for cat in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat],
                                                                              y_pred[:, num],
                                                                              average='weighted',
                                                                              labels=category_names)

        categories.append(cat)
        f_scores.append(f_score)
        precisions.append(precision)
        recalls.append(recall)

    results = pd.DataFrame({'Category':categories, 'f_score': f_scores, 'precision': precisions, 'recall':recalls})

    print('Aggregated f_score:', results['f_score'].mean())
    print('Aggregated precision:', results['precision'].mean())
    print('Aggregated recall:', results['recall'].mean())
    return results

In [13]:
performance_results(y_test, y_pred, category_names)

Aggregated f_score: 0.1949845629696122
Aggregated precision: 0.9358698665166619
Aggregated recall: 0.17263645224171542


/Users/pepe/anaconda3/envs/python3_new/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Category,f_score,precision,recall
0,related,0.758084,0.785314,0.795767
1,request,0.212009,0.829724,0.262395
2,offer,0.160645,0.994127,0.093249
3,aid_related,0.408311,0.756804,0.499047
4,medical_help,0.170683,0.922760,0.163806
5,medical_products,0.163720,0.952184,0.132532
6,search_and_rescue,0.161707,0.975166,0.112891
7,security,0.160584,0.983032,0.102021
8,military,0.161928,0.967859,0.118040
9,child_alone,0.160645,1.000000,0.087338


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a19537830>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                 

In [15]:
# parameters =  {'clf__estimator__n_estimators': [50, 100], 
#               'clf__estimator__min_samples_split': [2, 4],
#               'tfidf__use_idf': (True, False)} 

parameters = {'clf__estimator__max_depth': [10, 50, None],
                  'clf__estimator__min_samples_leaf': [2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
results2 = performance_results(y_test, y_pred, category_names)
results2

KeyboardInterrupt: 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [ ]:
parameters_2 = {'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2],
              'tfidf__use_idf': (True, False), }

In [ ]:
cv_2 = GridSearchCV(pipeline_2, param_grid=parameters_2)
cv_2

In [ ]:
cv_2.fit(X_train, y_train)

In [ ]:
y_pred = cv_2.predict(X_test)
results3 = performance_results(y_test, y_pred, category_names)
results3

### 9. Export your model as a pickle file

In [ ]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(cv_2, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.